In [2]:
import ee

In [3]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWhe67PPKom7nH9xPd3H8qi-lrcEuzoquiqwiLFwfTAumRQJk6Qw7O0



Successfully saved authorization token.


In [71]:
ee.Initialize()

In [4]:
import importlib
from flood_detection import modis
importlib.reload(modis)
from flood_detection.utils import misc
import geemap
from pathlib import Path
import rasterio

In [5]:
Rajshahi = ee.FeatureCollection("projects/ee-bobgiezi/assets/Gates/Rajshahi").geometry().bounds()
Kurigram = ee.FeatureCollection("projects/ee-bobgiezi/assets/Gates/Kurigram").geometry().bounds()
Barisal = ee.FeatureCollection("projects/ee-bobgiezi/assets/Gates/Barisal").geometry().bounds()

In [6]:
geoms = list()
geoms.append(Rajshahi)
geoms.append(Kurigram)
geoms.append(Barisal)

In [7]:
names = ['Rajshahi', 'Kurigram', 'Barisal']

In [10]:
for k in [2]:#range(len(geoms)):
    for year in range(2001, 2022):
        geom = geoms[k]
        name = names[k]
        
        filename = str(Path('DataExport')/name/(str(year)+'.tif'))

        flood_map = modis.dfo(geom, str(year)+"-06-01", str(year)+"-12-31", "standard", '3Day', True)

        flood_map_slope_mask = misc.apply_slope_mask(flood_map, thresh=5)

        perm_water = misc.get_jrc_perm(geom)

        dfo_final = ee.Image(flood_map_slope_mask).addBands(perm_water)

        geemap.ee_export_image(dfo_final,\
                           filename,\
                           scale=250,\
                           crs='EPSG:4326',\
                           region=geom,\
                           file_per_band=False)

Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Barisal/2001.tif
Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Barisal/2002.tif
Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Barisal/2003.tif
Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Barisal/2004.tif
Collected and pre-processed MODIS Images
DFO Flood Dectection Complete
Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/work/MODIS_GlobalFloodDatabase/DataExport/Barisal/2005.tif
Collected and p

In [11]:
for name in [names[2]]:
    for year in range(2001, 2022):
        with rasterio.open(Path('DataExport')/name/(str(year)+'.tif'),'r+') as dst:
            dst.descriptions = tuple(['flooded', 'duration', 'clearViews', 'clearPerc', 'maxExtent', 'permanentWater'])